# Imports

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns

from sklearn.model_selection import train_test_split 
from imblearn.over_sampling import RandomOverSampler


pd.set_option('display.float_format', lambda x: '%.2f' % x)
RSEED = 42

### Import Data

In [41]:
invoice_train = pd.read_csv('data/train/invoice_train.csv')
client_train = pd.read_csv('data/train/client_train.csv')


/var/folders/9x/vhdwn1fd0cldzwf9fnftwf980000gn/T/ipykernel_76133/3220789026.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  invoice_train = pd.read_csv('data/train/invoice_train.csv')


In [42]:
print(client_train.shape)
client_train.head()

(135493, 6)


,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.00
1,69,train_Client_1,11,107,29/05/2002,0.00
2,62,train_Client_10,11,301,13/03/1986,0.00
3,69,train_Client_100,11,105,11/07/1996,0.00
4,62,train_Client_1000,11,303,14/10/2014,0.00


In [43]:
invoice_train.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC


## EDA


### Drop Outliers

In [44]:
# Drop outliers high month
invoice_train= invoice_train[invoice_train['months_number']<=36]

In [45]:
# Drop invoices where month = 0
invoice_train = invoice_train[invoice_train['months_number']>0]


In [46]:
invoice_train.columns

Index(['client_id', 'invoice_date', 'tarif_type', 'counter_number',
       'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'old_index',
       'new_index', 'months_number', 'counter_type'],
      dtype='object')

### Calculate over Columns

#### encode counter_type
`counter_type`

In [47]:
# encode counter_type
d={"ELEC":0,"GAZ":1}
invoice_train['counter_type']=invoice_train['counter_type'].map(d)

#### Calculate Consommation sum over levels
`consommation_level_1`, `consommation_level_2`, `consommation_level_3`, `consommation_level_4`

In [48]:
# calculate consummation per time
# sum all consummation levels and devide by month_number
consommation_sum = (invoice_train['consommation_level_1']+invoice_train['consommation_level_2']+invoice_train['consommation_level_3']+invoice_train['consommation_level_4'])
invoice_train['consommation_sum'] = consommation_sum
invoice_train['consommation_per_month'] = consommation_sum/invoice_train['months_number']

In [49]:
invoice_train.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type,consommation_sum,consommation_per_month
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,0,82,20.50
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,0,1384,346.00
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,0,123,30.75
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,0,102,25.50
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,0,572,47.67


#### Create for categorical feature a new column: 
`reading_remarque`,`counter_statue`,`tarif_type`,`counter_coefficient`


In [50]:
invoice_train['counter_statue'].unique()

array([0, 1, 5, 4, 3, 2, '0', '5', '1', '4', 'A', 618], dtype=object)

In [51]:
# Define a mapping dictionary
mapping = {
    '0': 0,
    '1': 1,
    '5': 5,
    '4': 4,
    '3': 3,
    '2': 2,
    '46': 46,
    'A': 13,
    '618': 618,
    0:0,
    1:1,
    2:2,
    3:3,
    4:4,
    5:5,
    618:618
}

# Apply the mapping to the 'counter_statue' column
invoice_train['counter_statue'] = invoice_train['counter_statue'].map(mapping)

In [52]:
invoice_train['counter_statue'].unique()

array([  0,   1,   5,   4,   3,   2,  13, 618])

In [53]:
def OneHotEncodeAndAggregate(df, columns_to_encode):
    '''aggregated_df = OneHotEncodeAndAggregate(df, ['column1', 'column2', 'column3'])'''
    # Perform one-hot encoding for specified columns
    df_encoded = pd.get_dummies(df, columns=columns_to_encode, prefix=columns_to_encode)

    # Group by 'client_id' and sum the one-hot encoding columns
    aggregated_df = df_encoded.groupby('client_id').sum().reset_index()

    return aggregated_df

In [54]:
df = OneHotEncodeAndAggregate(invoice_train[['client_id','reading_remarque','counter_statue','tarif_type','counter_coefficient']], ['reading_remarque','counter_statue','tarif_type','counter_coefficient'])
df.head(3)

,client_id,reading_remarque_6,reading_remarque_7,reading_remarque_8,reading_remarque_9,reading_remarque_413,counter_statue_0,counter_statue_1,counter_statue_2,counter_statue_3,...,counter_coefficient_5,counter_coefficient_6,counter_coefficient_9,counter_coefficient_10,counter_coefficient_11,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50
0,train_Client_0,21,0,8,6,0,35,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,train_Client_1,20,0,6,11,0,37,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,train_Client_10,10,0,5,3,0,18,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
df.columns

Index(['client_id', 'reading_remarque_6', 'reading_remarque_7',
       'reading_remarque_8', 'reading_remarque_9', 'reading_remarque_413',
       'counter_statue_0', 'counter_statue_1', 'counter_statue_2',
       'counter_statue_3', 'counter_statue_4', 'counter_statue_5',
       'counter_statue_13', 'counter_statue_618', 'tarif_type_8',
       'tarif_type_9', 'tarif_type_10', 'tarif_type_11', 'tarif_type_12',
       'tarif_type_13', 'tarif_type_14', 'tarif_type_15', 'tarif_type_18',
       'tarif_type_21', 'tarif_type_24', 'tarif_type_27', 'tarif_type_29',
       'tarif_type_30', 'tarif_type_40', 'tarif_type_42', 'tarif_type_45',
       'counter_coefficient_0', 'counter_coefficient_1',
       'counter_coefficient_2', 'counter_coefficient_3',
       'counter_coefficient_4', 'counter_coefficient_5',
       'counter_coefficient_6', 'counter_coefficient_9',
       'counter_coefficient_10', 'counter_coefficient_11',
       'counter_coefficient_20', 'counter_coefficient_30',
       'counter_

#### Count unique values instead of categorical features
`counter_code`, `counter_number`, `counter_type`

In [56]:
def AggregateUniqueValues(df, group_column, columns_to_count):
    # Group by 'group_column' and calculate the number of unique values for specified columns
    unique_values = df.groupby(group_column)[columns_to_count].nunique()
    
    # Reset the index to make the result a DataFrame
    unique_values_df = unique_values.reset_index()
    
    # Rename the columns for clarity
    unique_values_df.columns = [group_column] + [f'{col}_count_unique' for col in columns_to_count]
    
    return unique_values_df

In [57]:
df2 = AggregateUniqueValues(invoice_train, 'client_id', ['counter_code', 'counter_number', 'counter_type'])
df2.head(10)

,client_id,counter_code_count_unique,counter_number_count_unique,counter_type_count_unique
0,train_Client_0,2,1,1
1,train_Client_1,1,1,1
2,train_Client_10,2,1,1
3,train_Client_100,1,1,1
4,train_Client_1000,1,1,1
5,train_Client_10000,2,2,2
6,train_Client_100000,2,2,2
7,train_Client_100001,2,2,2
8,train_Client_100002,1,1,1
9,train_Client_100003,1,1,1


#### Mean Consommation & Months
`consommation_sum`, `months_number`

In [58]:
df3 = invoice_train.groupby('client_id')['months_number'].mean().reset_index(name='mean_months')


In [59]:
df4 = invoice_train.groupby('client_id')['consommation_sum'].mean().reset_index(name='mean_consommation')


### Merge tables

In [60]:
merged_df = client_train.merge(invoice_train, how= "inner", on='client_id')

In [61]:
merged_df = merged_df.merge(df, how= "inner", on='client_id')
merged_df = merged_df.merge(df2, how= "inner", on='client_id')
merged_df = merged_df.merge(df3, how= "inner", on='client_id')
merged_df = merged_df.merge(df4, how= "inner", on='client_id')

In [62]:
merged_df.head(5)

,disrict,client_id,client_catg,region,creation_date,target,invoice_date,tarif_type,counter_number,counter_statue,...,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_code_count_unique,counter_number_count_unique,counter_type_count_unique,mean_months,mean_consommation
0,60,train_Client_0,11,101,31/12/1994,0.00,2014-03-24,11,1335667,0,...,0,0,0,0,0,2,1,1,4.63,362.97
1,60,train_Client_0,11,101,31/12/1994,0.00,2013-03-29,11,1335667,0,...,0,0,0,0,0,2,1,1,4.63,362.97
2,60,train_Client_0,11,101,31/12/1994,0.00,2015-03-23,11,1335667,0,...,0,0,0,0,0,2,1,1,4.63,362.97
3,60,train_Client_0,11,101,31/12/1994,0.00,2015-07-13,11,1335667,0,...,0,0,0,0,0,2,1,1,4.63,362.97
4,60,train_Client_0,11,101,31/12/1994,0.00,2016-11-17,11,1335667,0,...,0,0,0,0,0,2,1,1,4.63,362.97


In [63]:
merged_df.columns

Index(['disrict', 'client_id', 'client_catg', 'region', 'creation_date',
       'target', 'invoice_date', 'tarif_type', 'counter_number',
       'counter_statue', 'counter_code', 'reading_remarque',
       'counter_coefficient', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'old_index',
       'new_index', 'months_number', 'counter_type', 'consommation_sum',
       'consommation_per_month', 'reading_remarque_6', 'reading_remarque_7',
       'reading_remarque_8', 'reading_remarque_9', 'reading_remarque_413',
       'counter_statue_0', 'counter_statue_1', 'counter_statue_2',
       'counter_statue_3', 'counter_statue_4', 'counter_statue_5',
       'counter_statue_13', 'counter_statue_618', 'tarif_type_8',
       'tarif_type_9', 'tarif_type_10', 'tarif_type_11', 'tarif_type_12',
       'tarif_type_13', 'tarif_type_14', 'tarif_type_15', 'tarif_type_18',
       'tarif_type_21', 'tarif_type_24', 'tarif_type_27', 'tarif_type_29',
      

In [64]:
print(merged_df.isna().sum())

disrict                        0
client_id                      0
client_catg                    0
region                         0
creation_date                  0
                              ..
counter_code_count_unique      0
counter_number_count_unique    0
counter_type_count_unique      0
mean_months                    0
mean_consommation              0
Length: 73, dtype: int64


In [65]:
# msno.matrix(merged_df) # it took a while

### Tidy up & Drop Duplicates

In [66]:
# drop columns we dont aggregate
not_agg_features = ['invoice_date', 'creation_date','tarif_type', 'counter_number', 'counter_type', 'counter_coefficient', 'counter_statue', 'counter_code', 'reading_remarque', 'consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4', 'consommation_sum', 'consommation_per_month', 'old_index', 'new_index', 'months_number' ]
merged_df = merged_df.drop(not_agg_features, axis=1)

In [67]:
merged_df.sort_values('client_id').head()

,disrict,client_id,client_catg,region,target,reading_remarque_6,reading_remarque_7,reading_remarque_8,reading_remarque_9,reading_remarque_413,...,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_code_count_unique,counter_number_count_unique,counter_type_count_unique,mean_months,mean_consommation
0,60,train_Client_0,11,101,0.00,21,0,8,6,0,...,0,0,0,0,0,2,1,1,4.63,362.97
20,60,train_Client_0,11,101,0.00,21,0,8,6,0,...,0,0,0,0,0,2,1,1,4.63,362.97
21,60,train_Client_0,11,101,0.00,21,0,8,6,0,...,0,0,0,0,0,2,1,1,4.63,362.97
22,60,train_Client_0,11,101,0.00,21,0,8,6,0,...,0,0,0,0,0,2,1,1,4.63,362.97
23,60,train_Client_0,11,101,0.00,21,0,8,6,0,...,0,0,0,0,0,2,1,1,4.63,362.97


In [68]:
# delete duplicates
merged_df = merged_df.drop_duplicates()
merged_df.sort_values('client_id').head()

,disrict,client_id,client_catg,region,target,reading_remarque_6,reading_remarque_7,reading_remarque_8,reading_remarque_9,reading_remarque_413,...,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_code_count_unique,counter_number_count_unique,counter_type_count_unique,mean_months,mean_consommation
0,60,train_Client_0,11,101,0.00,21,0,8,6,0,...,0,0,0,0,0,2,1,1,4.63,362.97
35,69,train_Client_1,11,107,0.00,20,0,6,11,0,...,0,0,0,0,0,1,1,1,4.32,557.54
72,62,train_Client_10,11,301,0.00,10,0,5,3,0,...,0,0,0,0,0,2,1,1,6.44,836.50
90,69,train_Client_100,11,105,0.00,19,0,0,1,0,...,0,0,0,0,0,1,1,1,4.20,1.20
110,62,train_Client_1000,11,303,0.00,0,0,2,12,0,...,0,0,0,0,0,1,1,1,3.71,922.64


In [69]:
merged_df.isnull().sum()

disrict                        0
client_id                      0
client_catg                    0
region                         0
target                         0
reading_remarque_6             0
reading_remarque_7             0
reading_remarque_8             0
reading_remarque_9             0
reading_remarque_413           0
counter_statue_0               0
counter_statue_1               0
counter_statue_2               0
counter_statue_3               0
counter_statue_4               0
counter_statue_5               0
counter_statue_13              0
counter_statue_618             0
tarif_type_8                   0
tarif_type_9                   0
tarif_type_10                  0
tarif_type_11                  0
tarif_type_12                  0
tarif_type_13                  0
tarif_type_14                  0
tarif_type_15                  0
tarif_type_18                  0
tarif_type_21                  0
tarif_type_24                  0
tarif_type_27                  0
tarif_type

In [71]:
# clean up columns for modeling

merged_df = merged_df.rename(columns={'disrict': 'district',   'mean_consommation': 'consommation_sum',  'mean_months': 'months_number'})


In [72]:
merged_df.head(2)

,district,client_id,client_catg,region,target,reading_remarque_6,reading_remarque_7,reading_remarque_8,reading_remarque_9,reading_remarque_413,...,counter_coefficient_20,counter_coefficient_30,counter_coefficient_33,counter_coefficient_40,counter_coefficient_50,counter_code_count_unique,counter_number_count_unique,counter_type_count_unique,months_number,consommation_sum
0,60,train_Client_0,11,101,0.00,21,0,8,6,0,...,0,0,0,0,0,2,1,1,4.63,362.97
35,69,train_Client_1,11,107,0.00,20,0,6,11,0,...,0,0,0,0,0,1,1,1,4.32,557.54


### Train-Test-Split

In [73]:
y = merged_df['target']
X = merged_df.drop('target', axis = 1)

In [74]:
# test train split only with client-data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RSEED, stratify=y)

In [75]:
X_train['target'] = y_train

In [76]:
X_test['target'] = y_test

In [77]:
X_train.shape

(101574, 55)

In [78]:
X_test.shape

(33859, 55)

### Safe dataframes

In [79]:
X_train.to_csv('data/cleaned/train_extended.csv', index=False)

In [80]:
X_test.to_csv('data/cleaned/test_extended.csv', index=False)